Q6. Quora question pair similarity, you need to find the Similarity between two
questions by mapping the words in the questions using TF-IDF, and using a supervised
Algorithm you need to find the similarity between the questions

In [1]:
import shutil
import requests
import pandas as pd
pd.options.display.max_columns = None

r = requests.get('https://storage.googleapis.com/kagglesdsdata/competitions/6277/323734/train.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1686305674&Signature=TP71Od9Hp4WAo%2FsLT9R9ENCXv3fAEynbCrddUQ0zx%2Fceg5w2teFsj1Ml6gejavF65Xv4AjBj7%2FL8vpdgtFXRAYfGaVsmCineGVoanmUmoMYPnsY3jkZYSG2rJrGyiN7F4BBJ%2FTdz7ouBN94GyuO4h7u11TRpiY8eKYFJd2rvjEpGrXn2DEok1l6H%2BkqLXK8wbnB3SZXM4hJoQH8%2Bh5g1jlqltITaylec7x%2FuvFAI21PNTwyQG1ZUItXXtbFqzc8PK6lRXIBtixSbNWM8YxWXZtGXVrNgnA96rJeti8D4Iii9cKeg0yylDpYsYWsa0MU5d7f4XUo1uAF3%2BcGG7s2ZQA%3D%3D&response-content-disposition=attachment%3B+filename%3Dtrain.csv.zip')
open('quora.csv.zip', 'wb').write(r.content)


shutil.unpack_archive('quora.csv.zip')
data = pd.read_csv('train.csv')

In [2]:
data.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [3]:
data = data.iloc[:3000,:]

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            3000 non-null   int64 
 1   qid1          3000 non-null   int64 
 2   qid2          3000 non-null   int64 
 3   question1     3000 non-null   object
 4   question2     3000 non-null   object
 5   is_duplicate  3000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 140.8+ KB


In [5]:
data.isna().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [6]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

def stem(text):
  y=[]
  for i in text.split():
    y.append(ps.stem(i))
  return ' '.join(y)

In [7]:
data['question1'] = data['question1'].apply(stem)
data['question2'] = data['question2'].apply(stem)

In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data[['question1', 'question2']], data['is_duplicate'], test_size=0.2, random_state=42)

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the training data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train['question1'] + ' ' + X_train['question2'])

# Transform the test data
X_test_tfidf = tfidf_vectorizer.transform(X_test['question1'] + ' ' + X_test['question2'])


In [9]:
# Initialize the logistic regression model
model = LogisticRegression()

# Train the model
model.fit(X_train_tfidf, y_train)

# Predict on the test set
y_pred = model.predict(X_test_tfidf)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"LogisticRegression Accuracy: {accuracy}")

LogisticRegression Accuracy: 0.6683333333333333
